In [1]:
#libraries
import numpy as np 
import pandas as pd 

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.model_selection import cross_val_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mlcourse-dota2-win-prediction/sample_submission.csv
/kaggle/input/mlcourse-dota2-win-prediction/train_matches.jsonl
/kaggle/input/mlcourse-dota2-win-prediction/test_matches.jsonl
/kaggle/input/mlcourse-dota2-win-prediction/train_features.csv
/kaggle/input/mlcourse-dota2-win-prediction/test_features.csv
/kaggle/input/mlcourse-dota2-win-prediction/train_targets.csv


In [2]:
%%time
PATH_TO_DATA = '../input/mlcourse-dota2-win-prediction'
df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA,'train_features.csv'), index_col='match_id_hash')
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_targets.csv'), index_col='match_id_hash')

CPU times: user 1.31 s, sys: 193 ms, total: 1.5 s
Wall time: 1.91 s


In [3]:
df_train_features.shape, df_train_targets.shape

((39675, 245), (39675, 5))

In [7]:
df_train_features.head()
df_train_features.tail()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
defd0caeed6ea83d7d5fbdec013fe7d1,1783,22,0,8,23,3,1,9,9,0,...,26.493834,6,2,3,0,0.5625,0,0,7,2
bc7a87ed5f9c2bca55f9f7a93da0b0c5,377,22,7,1,0,14,0,2,1,1,...,0.000000,0,0,1,0,0.0000,0,0,0,0
e2ca68ac1a6847f4a37f6c9c8ee8695b,643,22,7,1,23,63,1,4,0,2,...,2.532715,0,0,3,1,0.5000,0,0,1,0
47ad6454ede66c1c78fdaa9391dfc556,2405,22,7,12,4,22,3,8,14,7,...,17.583984,3,1,4,0,0.6250,0,0,0,0
9928dfde50efcbdb2055da23dcdbc101,1775,22,0,10,13,32,3,4,15,0,...,8.331238,0,0,1,0,0.5500,1,0,0,0


In [5]:
df_train_targets.head()

,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,False,992,837,NaN
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
46a0ddce8f7ed2a8d9bd5edcbb925682,576,True,1952,1376,NaN
b1b35ff97723d9b7ade1c9c3cf48f770,453,False,2001,1548,NaN


In [11]:
X = df_train_features.values
y = df_train_targets['radiant_win'].values

print(X)
print(y)

[[1.550e+02 2.200e+01 7.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [6.580e+02 4.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.100e+01 2.300e+01 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [6.430e+02 2.200e+01 7.000e+00 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.405e+03 2.200e+01 7.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.775e+03 2.200e+01 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]
[False  True  True ... False  True  True]


In [16]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3,random_state=17)

In [17]:
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((27772, 245), (11903, 245), (27772,), (11903,))

In [18]:
np.bincount(y)

array([18849, 20826])

In [20]:

C = 1
penalty = 'l2'
max_iter = 100
solver = 'liblinear'
random_state = 17
n_jobs = -1
verbose = 1

clf_lr = LogisticRegression(C=C,
                            penalty=penalty,
                            max_iter=max_iter, 
                            random_state=random_state,
                            verbose=verbose,
                            n_jobs=n_jobs,
                           solver=solver)

In [22]:
%%time
clf_lr.fit(x_train, y_train)
y_pred = clf_lr.predict(x_valid)
print('Log Regression validation roc_auc score {} '.format(roc_auc_score(y_pred, y_valid)))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]Log Regression validation roc_auc score 0.7266546207314954 
CPU times: user 3min 47s, sys: 16.7 ms, total: 3min 47s
Wall time: 3min 47s


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
